# structure
- get all page
    - get article
        - get content
        - get image
        - OCR

## data format
- type wrong | partial wrong | correct
- title
- content
- category
- time
- report

### update teseract v5 ppa

In [9]:
# !rm /opt/conda/lib/libcurl.so.4
# !ln -s /usr/lib/x86_64-linux-gnu/libcurl.so.4.8.0 /opt/conda/lib/libcurl.so.4
!add-apt-repository ppa:alex-p/tesseract-ocr5 -y
!apt update
!apt install -y tesseract-ocr
!apt install tesseract-ocr-eng
# If you want all languages -- please note this will take time to download. You could also point to the already install tessdata in 4.00 as well.
# !apt install tesseract-ocr-all -y
!tesseract --version

ln: failed to create symbolic link '/opt/conda/lib/libcurl.so.4': No such file or directory
Repository: 'deb https://ppa.launchpadcontent.net/alex-p/tesseract-ocr5/ubuntu/ jammy main'
Description:
 The Tesseract OCR engine was one of the top 3 engines in the 1995
 UNLV Accuracy test. Between 1995 and 2006 it had little work done on
 it, but since then it has been improved extensively by Google and is
 probably one of the most accurate open source OCR engines
 available. It can read a wide variety of image formats and convert
 them to text in over 40 languages. This package includes the command
 line tool.
More info: https://launchpad.net/~alex-p/+archive/ubuntu/tesseract-ocr5
Adding repository.
Adding deb entry to /etc/apt/sources.list.d/alex-p-ubuntu-tesseract-ocr5-jammy.list
Adding disabled deb-src entry to /etc/apt/sources.list.d/alex-p-ubuntu-tesseract-ocr5-jammy.list
Adding key to /etc/apt/trusted.gpg.d/alex-p-ubuntu-tesseract-ocr5.gpg with fingerprint 8529B1E0F8BF7F65C12FABB0A4BC

### install teseract-ocr v5

In [10]:
!sudo apt install tesseract-ocr-chi-tra
!pip install pytesseract

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following packages will be upgraded:
  tesseract-ocr-chi-tra
1 upgraded, 0 newly installed, 0 to remove and 47 not upgraded.
Need to get 1,646 kB of archives.
After this operation, 0 B of additional disk space will be used.
Get:1 https://ppa.launchpadcontent.net/alex-p/tesseract-ocr5/ubuntu jammy/main amd64 tesseract-ocr-chi-tra all 1:5.0.0~git39-6572757-2ppa1~jammy1 [1,646 kB]
Fetched 1,646 kB in 1s (1,140 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
(Reading database ... 121844 file

In [69]:
!tesseract --version | cat
!tesseract --list-langs | cat

tesseract 5.3.4
 leptonica-1.82.0
  libgif 5.1.9 : libjpeg 8d (libjpeg-turbo 2.1.1) : libpng 1.6.37 : libtiff 4.3.0 : zlib 1.2.11 : libwebp 1.2.2 : libopenjp2 2.4.0
 Found AVX2
 Found AVX
 Found FMA
 Found SSE4.1
 Found OpenMP 201511
 Found libarchive 3.6.0 zlib/1.2.11 liblzma/5.2.5 bz2lib/1.0.8 liblz4/1.9.3 libzstd/1.4.8
 Found libcurl/7.81.0 OpenSSL/3.0.2 zlib/1.2.11 brotli/1.0.9 zstd/1.4.8 libidn2/2.3.2 libpsl/0.21.0 (+libidn2/2.3.2) libssh/0.9.6/openssl/zlib nghttp2/1.43.0 librtmp/2.3 OpenLDAP/2.5.16
List of available languages in "/usr/share/tesseract-ocr/5/tessdata/" (2):
chi_tra
eng


### import

In [93]:
import pytesseract
import os,re
try:
    from PIL import Image
except ImportError:
    import Image
from bs4 import BeautifulSoup
import requests

### get all linlks

In [94]:
# url = 'https://tfc-taiwan.org.tw/articles/report'
site_url = 'https://tfc-taiwan.org.tw'
# url = f'{site_url}/articles/9897'
url = f'{site_url}/articles/10481'
res = requests.get(url)
res.encoding = 'utf-8'
html_doc = res.text
soup = BeautifulSoup(html_doc, 'html.parser')

In [172]:
def get_pages():
    url = "https://tfc-taiwan.org.tw/articles/report?page="
    done = False
    i = 0
    links = []
    while not done:
        res = requests.get(f"{url}{i}")
        res.encoding = 'utf-8'
        if res.status_code != 200:
            done = True
        else:
            print(i,len(links))
            tmp,done =  get_links(res.text)
            links.extend(tmp)
        i += 1

def get_links(html_doc):
    soup = BeautifulSoup(html_doc, 'html.parser')
    pattern = re.compile(r'^/articles/\d+')
    page = soup.select(".moscone-flipped-content")#[0].select(".view-content")
    # page = soup.select(".moscone-flipped-column-content-region")
    print(len(page))
    links = page[0].find_all("a",href=pattern)
    print(len(links))
    links = [i.get('href') for i in links]
    links = list(set(links))
    print(links)
    if len(links) == 0:
        return [],True
    else:
        return links,False

# url = "https://tfc-taiwan.org.tw/articles/report?page=295"
# res = requests.get(url)
# res.encoding = 'utf-8'
# get_links(res.text)
get_pages()


290 0
1
30
['/articles/335', '/articles/350', '/articles/356', '/articles/315', '/articles/349', '/articles/336', '/articles/343', '/articles/326', '/articles/307', '/articles/355']
291 10
1
30
['/articles/296', '/articles/275', '/articles/305', '/articles/288', '/articles/276', '/articles/292', '/articles/280', '/articles/267', '/articles/289', '/articles/304']
292 20
1
30
['/articles/237', '/articles/217', '/articles/206', '/articles/261', '/articles/238', '/articles/250', '/articles/203', '/articles/260', '/articles/249', '/articles/223']
293 30
1
30
['/articles/150', '/articles/149', '/articles/199', '/articles/164', '/articles/189', '/articles/183', '/articles/188', '/articles/173', '/articles/148', '/articles/171']
294 40
1
30
['/articles/105', '/articles/106', '/articles/128', '/articles/104', '/articles/142', '/articles/136', '/articles/112', '/articles/111', '/articles/123', '/articles/131']
295 50
1
14
['/articles/100', '/articles/83', '/articles/98', '/articles/84', '/articl

In [131]:
title = soup.select(".entity-list-title")[0].text
title

'網傳「接到陌生電話不要先講話，不出聲要直接掛掉，AI會蒐集民眾聲音」?'

In [82]:
reply =  soup.select(".node-preface")[0].text
reply

'經查：【報告將隨時更新 2024/4/12版】\nAI技術日新月異，近期通訊軟體流傳一段文字訊息，提醒民眾接到陌生電話不要先講話，因為AI會蒐集民眾聲音。專家看法為何：\n一、AI專家指出，目前AI技術發展快，大約5-10秒間的連續說話，能模擬出真實人聲。但實際電話通話情境，聲音品質可能不佳、容易失真；且若只是幾句「喂、你好」回應，難以製作出聲音品質佳的AI音檔。\n專家建議，民眾不需要過度防備，也不需要接起電話先不出聲。\n二、資安、AI專家說，對於陌生電話本來就應該小心謹慎，若擔心可能是詐騙電話，應直接掛掉，謹慎求證。民眾也可以跟家人朋友間設立「通話密語」，在接電話時，若不確定是否為親友本人時就能進行多重確認。\n'

In [83]:

page = soup.select(".content-inner")[0]
imgs = page.find_all('img')
src = [i['src'] for i in imgs]
# for i in imgs:
#     print(i['src'])
#     src = i['src']


In [84]:
src

['/sites/default/files/drive_api/20240412_175505_a0340fb8ad24b72b1aa8ae58d23514c3_______ 2024-04-11 112937.png',
 '/sites/default/files/drive_api/20240412_175506_5f26f3cb499347e989b64cba2d30ce49_S__37576756.jpg']

In [73]:
def ocr(image_path):
    image = Image.open(image_path)
    text = pytesseract.image_to_string(image, lang='chi_tra')
    print(text)
    return text

In [123]:
for i in src:
    res = requests.get(f"{site_url}{i}")
    if res.status_code == 200:
        with open('test.png', 'wb') as f:
            f.write(res.content)
    ocr("test.png")
    print("============================================")

無聲來電"喂一聲"聲音被
偷?! Al"5秒複製人聲'驚

p ooz/i53 一一一                人間 呈

這是一

上

很重要的訊息,當你

接到電話記住不要先出聲

如果對方還是不出聲,就要掛
斷

切記!切記!因為Al正在收集
你的聲音

14:16



In [8]:
text

" 人訪詩還計時廬生主和3和1生主:本六証廬還號1t計計二還還品全共生生 全還點=中記歡人邊中:人員\n停電,近2999多戶受到影響,對此,台電正派員搶修當中。另北市貓空纜車也受到影響暫停營運。\n\n \n\n共和EE和還汪e廬近還2自廬lt刻本主引必2二信二中加放生中全和中1本莒1汪由5廬1\n| 息,而瞬間大雷雨也讓台北市士林區、淡水區發生停電,致福國路、福華路忠誠路一段受到影\n\n \n\n' 對此,台電表示,事發後台電隨即派員至現場查修,於下午已復電約7成(1282戶)正常供電,台電現仍持續搶修中,目前仍尋找故\n| 障點中。另北市貓空纜車也受到影響暫停營運,提醒旅客勿前往搭乘。\n原文網址: 雙北降豪兩 !士林、淡水大規模停電 貓纜暫停搭乘 | 生活 | Nawtalk新聞 https://newtalk.tw/news/view/\n避全EE Ki\nFollow us:; @Newtalk新聞 on FB | Qnewtalk_news on IG | “@NewtaukNews on YT\n\n \n\n \n\n \n\n \n\x0c"

In [3]:
os.environ['TESSDATA_PREFIX']

KeyError: 'TESSDATA_PREFIX'